# 2. Data Cleaning

This notebook contains code which reads in area/route information gathered during the scraping process
and cleans/processes it so it can analyzed and input into the recommender system algorithm.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import pickle
import warnings

import requests
from bs4 import BeautifulSoup
import functools
import operator
import os
from MPAreaTree import MPAreaTree
pd.set_option('display.max_colwidth', 4000)
warnings.filterwarnings('ignore')

In [2]:
# Read in the boulder and trad/sport climb grades
boulder_grades = pd.read_csv('./data/boulder_grades.csv')
climb_grades = pd.read_csv('./data/climb_grades.csv')


# Create a dictionary of the grades
# adding an ordinal component so we can sort routes by difficulty 
grade_dict = {v:k for k,v in boulder_grades.to_dict()['grade'].items()}
grade_dict.update({v:k for k,v in climb_grades.to_dict()['grade'].items()})


# Clean and format the route descriptions, IDs, names, height, containing area, rating etc. 
def clean_routes(routes, drop_unrated = True):
    cleaned_routes = routes.drop_duplicates()
    
    if drop_unrated:
        cleaned_routes = cleaned_routes[~cleaned_routes.star_ratings.isnull()]
        cleaned_routes = cleaned_routes[cleaned_routes['star_ratings'] != '{}']
        
    cleaned_routes['description'] = cleaned_routes['description'].fillna('')
    cleaned_routes['id'] = cleaned_routes['id'].astype(int)
    cleaned_routes = cleaned_routes.rename(columns = {'id' : 'route_id', 'name' : 'route_name'})
    cleaned_routes['area_id'] = cleaned_routes['area_id'].astype(int)
    cleaned_routes['pitches'] = cleaned_routes['pitches'].astype(int)
    cleaned_routes['votes'] = cleaned_routes['votes'].astype(int)
    cleaned_routes['route_name'] = cleaned_routes['route_name'].fillna('Unnamed')
    cleaned_routes['height'] = cleaned_routes['height'].map(lambda x: 'Unspecified' if x == 0 else x)
    # dictionary of user ratings is saved as a string, convert to a python dict
    cleaned_routes['star_ratings'] = cleaned_routes['star_ratings'].map(lambda x: ast.literal_eval(x))
       
    
    # remove ungraded routes for now
    cleaned_routes = cleaned_routes[~cleaned_routes['grade'].isin(['5.?', 'V?', ''])]
    cleaned_routes = cleaned_routes[cleaned_routes['grade'].isin(grade_dict.keys())]
    
    cleaned_routes = cleaned_routes[cleaned_routes['pitches'] != 80]
    cleaned_routes['grade_numeric'] = cleaned_routes['grade'].map(grade_dict).astype(int)
    return cleaned_routes

# Clean an areas information
def clean_area(areas):
    cleaned_areas = areas.drop_duplicates()
    
    cleaned_areas['parent_id'] = cleaned_areas['parent_id'].fillna(0)
    cleaned_areas['parent_id'] = cleaned_areas['parent_id'].astype(int)
    cleaned_areas = cleaned_areas.rename(columns = {'id' : 'area_id', 'name' : 'area_name'})
    return cleaned_areas

In [3]:
# traverse the rating dictionary for a dataframe of routes
# extract the user/route IDs and corresponding rating
# this generates the main user/route rating matrix used in the recommender system algorithm
def get_ratings(routes):
    ratings = []
    for i,row in routes.iterrows():
        for user, stars in row['star_ratings'].items():
            ratings.append({'user_id': user, 'route_id' : row['route_id'], 'user_rating' : stars})

    ratings = pd.DataFrame(ratings)
    
    return pd.merge(ratings, routes, left_on = 'route_id', right_on='route_id').drop(columns = 'star_ratings')

In [4]:
# Load in each states area/route information collected in step 1 of the scraping process
def load_state_routes_areas(states):
    routes = []
    areas = []
    for state in states:
        if os.path.exists(f'./data/{state}_routes.csv') and os.path.exists(f'./data/{state}_areas.csv'):
            print(f'Found route/area data for {state}.')
            r = clean_routes(pd.read_csv(f'./data/{state}_routes.csv',quoting=2, error_bad_lines=False))        
            a = clean_area(pd.read_csv(f'./data/{state}_areas.csv'))

            r['state'] = state
            a['state'] = state
            r = r.merge(a[['area_id', 'area_name']], left_on = 'area_id', right_on = 'area_id')
            routes.append(r)
            areas.append(a)            
    
    routes = pd.concat(routes).reset_index(drop = True)
    areas = pd.concat(areas).reset_index(drop = True)
    print(f'Loaded route/area data for {areas["state"].unique().shape[0]} states.')
    print(f"Found {routes['route_id'].unique().shape[0]} unique routes in {areas['area_id'].unique().shape[0]} unique areas.")
    return routes, areas

In [5]:
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", 
               "Delaware", "Florida", "Georgia",  "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas",
               "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", 
               "Mississippi", "Montana", "North Carolina",  "North Dakota", "Nebraska", "New Hampshire", 
               "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", 
               "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
               "Tennessee", "Texas", "Utah", "Virginia",  "Vermont", 
               "Washington", "Wisconsin", "West Virginia", "Wyoming"]
routes, areas = load_state_routes_areas(state_names)

Found route/area data for Alaska.
Found route/area data for Alabama.
Found route/area data for Arkansas.
Found route/area data for Arizona.
Found route/area data for California.
Found route/area data for Colorado.
Found route/area data for Connecticut.
Found route/area data for Delaware.
Found route/area data for Florida.
Found route/area data for Georgia.
Found route/area data for Hawaii.
Found route/area data for Iowa.
Found route/area data for Idaho.
Found route/area data for Illinois.
Found route/area data for Indiana.
Found route/area data for Kansas.
Found route/area data for Kentucky.
Found route/area data for Louisiana.
Found route/area data for Massachusetts.
Found route/area data for Maryland.
Found route/area data for Maine.
Found route/area data for Michigan.
Found route/area data for Minnesota.
Found route/area data for Missouri.
Found route/area data for Mississippi.
Found route/area data for Montana.
Found route/area data for North Carolina.
Found route/area data for Nor

In [6]:
# Generate the MPAreaTree (see associated python file) and the rating/route matrix
area_tree = MPAreaTree(areas)
ratings = get_ratings(routes)

In [7]:
ratings.shape

(2166284, 15)

In [38]:
# save all cleaned dataframes
routes.to_csv('./data/routes.csv', index = False)
areas.to_csv('./data/areas.csv', index = False)
ratings.to_csv('./data/ratings.csv', index = False)

In [39]:
# pickle the area tree object for use in the streamlight demo
pickle.dump(area_tree, open('./pickle/area_tree.pkl', 'wb'))